In [1]:
import pandas as pd
import numpy as np
from utils import *
from ReactNode import ReactNode
from ChemNode import ChemNode
from MCTS import MCTS
from tree_utils import *

In [2]:
conn = sqlite3.connect('data/split.db')
buyable = conn.cursor()
retrobiocat = pd.read_pickle("data/final_test_retrobiocat.pkl")
analyzer = Retrosim('data/RHEA_atom_mapped_timepoint_7_success.pkl', 'data/retrobiocat_database.pkl')

In [3]:
smile = "C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O"
root = ChemNode(smile, 0, None, buyable, retrobiocat, analyzer, None)

In [4]:
mcts = MCTS(root)

In [5]:
mcts.MCTS(5000)

100%|██████████| 5000/5000 [24:01<00:00,  3.47it/s] 


In [6]:
for react in root.reactions:
    print(f'Reaction name: {react.reaction_name}, Score: {react.get_mcts_value()}')
    for precursor in react.precursors:
        print(f'Chem {precursor.smiles}, Score: {precursor.score}, visits: {precursor.visits}, solution: {precursor.solution}')
    print(" ")

Reaction name: Ene hydrolysis, Score: 1.0135294970453652
Chem C#C[C@@]1(CO)C=C[C@H](n2cnc3c(N)nc(F)nc32)O1, Score: 105.57115500000033, visits: 156, solution: True
 
Reaction name: PNP, Score: 1.8209077138742555
Chem C#C[C@]1(CO)O[C@@H](OP(=O)([O-])O)C[C@@H]1O, Score: 538.5685686874979, visits: 801, solution: False
Chem Nc1nc(F)nc2[nH]cnc12, Score: 1.0, visits: 1, solution: True
 
Reaction name: 23454, Score: 0.8701232455435273
Chem C#C[C@]1(COP(=O)([O-])[O-])O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 346.4356512499984, visits: 507, solution: False
 
Reaction name: 28192, Score: 1.8038645220534344
Chem C#C[C@]1(CO)O[C@@H](n2cnc3c(=O)[nH]c(F)nc32)C[C@@H]1O, Score: 650.2773396875032, visits: 972, solution: True
Chem [NH4+], Score: 1.0, visits: 1, solution: True
 
Reaction name: 29372, Score: 1.824583690220249
Chem C#C[C@]1(COP(=O)([O-])[O-])O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 529.5154771874992, visits: 785, solution: False
Chem O, Score: 1.0, visits: 1, solution: True
 
React

In [24]:
root.solution

True

In [30]:
# Prune the tree starting from the root
# prune_tree(root)
paths = generate_paths(root)
print(len(paths))

5


In [26]:
path_explorer(paths[0])

BRANCH 0:
-----------------
Chem: C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O
Reaction: Ene hydrolysis
Chem: C#C[C@@]1(CO)C=C[C@H](n2cnc3c(N)nc(F)nc32)O1
Reaction: 27744
SPLIT: Chem1: C#C[C@@]1(CO)C=C[C@@H](OP(=O)([O-])[O-])O1, Chem2: Nc1nc(F)nc2[nH]cnc12

Reaction: 16038
SPLIT: Chem1: C#C[C@@]1(CO)C=C[C@H](n2cc(C)c(=O)[nH]c2=O)O1, Chem2: O=P([O-])([O-])O

BUYABLE

BRANCH 1:
-----------------
Chem: O=P([O-])([O-])O
BUYABLE

BRANCH 2:
-----------------
Chem: Nc1nc(F)nc2[nH]cnc12
BUYABLE



In [31]:
for i in range(0, len(paths)):
    print(f'Path {i}')
    path_explorer(paths[i])
    print(" ")

Path 0
BRANCH 0:
-----------------
Chem: C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O
Reaction: Ene hydrolysis
Chem: C#C[C@@]1(CO)C=C[C@H](n2cnc3c(N)nc(F)nc32)O1
Reaction: 27744
SPLIT: Chem1: C#C[C@@]1(CO)C=C[C@@H](OP(=O)([O-])[O-])O1, Chem2: Nc1nc(F)nc2[nH]cnc12

Reaction: 16038
SPLIT: Chem1: C#C[C@@]1(CO)C=C[C@H](n2cc(C)c(=O)[nH]c2=O)O1, Chem2: O=P([O-])([O-])O

BUYABLE

BRANCH 1:
-----------------
Chem: O=P([O-])([O-])O
BUYABLE

BRANCH 2:
-----------------
Chem: Nc1nc(F)nc2[nH]cnc12
BUYABLE

 
Path 1
BRANCH 0:
-----------------
Chem: C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O
Reaction: 28192
SPLIT: Chem1: C#C[C@]1(CO)O[C@@H](n2cnc3c(=O)[nH]c(F)nc32)C[C@@H]1O, Chem2: [NH4+]

Reaction: 27752
SPLIT: Chem1: C#C[C@]1(CO)O[C@H](OP(=O)([O-])[O-])C[C@@H]1O, Chem2: O=c1[nH]c(F)nc2nc[nH]c12

Reaction: 16038
SPLIT: Chem1: C#C[C@]1(CO)O[C@@H](n2cc(C)c(=O)[nH]c2=O)C[C@@H]1O, Chem2: O=P([O-])([O-])O

Reaction: Ene hydrolysis
Chem: C#C[C@@]1(CO)C=C[C@H](n2cc(C)c(=O)[nH]c2=O)O1
BUYABLE


In [17]:
canonicalize_smiles("C#C[C@@]1(CO)C=C[C@@H](OP(=O)([O-])[O-])O1")

'C#C[C@@]1(CO)C=C[C@@H](OP(=O)([O-])[O-])O1'

In [ ]:
check_in("C#C[C@]1(CO)O[C@H](OP(=O)([O-])[O-])C[C@@H]1O", buyable, 'buyable')

False

In [4]:
analyzer.single_step_retro(" C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1O", 10, False, True)

([('RHEA',
   41321,
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1OC(=O)CCCCCCC/C=C\\CCCCCCCC',
   0.4999223723024376),
  ('RHEA',
   62485,
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1OC(=O)CCCCCCCCCCCCCCC',
   0.49304677623261695),
  ('RHEA',
   40828,
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1OC(=O)CCC/C=C\\C/C=C\\C/C=C\\C/C=C\\CCCCC',
   0.4528301886792453),
  ('RetroBioCat',
   'Ene hydrolysis',
   'C#C[C@@]1(COP(=O)(O)O)C=C[C@H](OP(=O)(O)OCC[N+](C)(C)C)O1',
   0),
  ('RetroBioCat',
   'Ester hydrolysis (OH)',
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1OC(=O)CC',
   0),
  ('RetroBioCat',
   'Ketone reduction',
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)CC1=O',
   0),
  ('RetroBioCat',
   'N-methylation',
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[NH+](C)C)C[C@@H]1O',
   0),
  ('RetroBioCat',
   'Phosphate hydrolysis',
   'C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@

In [7]:
test = ChemNode("C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1O", 0, None, buyable, retrobiocat, analyzer, None)
test.possible_reactions = []
test.generate_reactions_retrobiocat()

for react in test.possible_reactions:
    print(f'Reaction name: {react.name}')
    for precursor in react.precursors:
        print(f'Chem {precursor}')
    print(" ")

Reaction name: Ene hydrolysis
Chem C#C[C@@]1(COP(=O)(O)O)C=C[C@H](OP(=O)(O)OCC[N+](C)(C)C)O1
 
Reaction name: Ester hydrolysis (OH)
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1OC(=O)CC
 
Reaction name: Ketone reduction
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)CC1=O
 
Reaction name: N-methylation
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[NH+](C)C)C[C@@H]1O
 
Reaction name: Phosphate hydrolysis
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1OP(=O)(O)O
 
Reaction name: Phosphorylation
Chem C[N+](C)(C)CCO
 
Reaction name: Phosphorylation
Chem C#C[C@]1(CO)O[C@@H](OP(=O)(O)OCC[N+](C)(C)C)C[C@@H]1O
 
Reaction name: Phosphorylation
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](O)C[C@@H]1O
 
